In [14]:
import numpy as np
import scipy.sparse as sp
import scipy
# import tensorflow as tf
import os
import multiprocessing
import torch

from collections import Counter
import json
import pickle
from tqdm import tqdm
import os
# import keras
import numpy as np
from utils import *
from tqdm import *
from evaluate import evaluate
import tensorflow as tf
# import keras.backend as K
# from keras.layers import *
from models import *
import torch.nn.functional as F
import torch
from models_encodercat import Encoder_Model
import random

In [3]:
alignment_pair = [
    (1, 101), (2, 102), (3, 103), (4, 104), (5, 105),
    (6, 106), (7, 107), (8, 108), (9, 109), (10, 110)
]


In [4]:
train_ratio = 0.3
val_rate = 0.1
np.random.shuffle(alignment_pair)

In [6]:
int(len(alignment_pair)*round(train_ratio-val_rate,1))


2

In [8]:
int(len(alignment_pair)*train_ratio)

3

In [9]:
int(len(alignment_pair)*train_ratio)

3

In [10]:
def normalize_adj(adj):
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return d_mat_inv_sqrt.dot(adj).transpose().dot(d_mat_inv_sqrt).T

def load_triples(file_name):
    triples = []
    entity = set()
    rel = set([0])
    for line in open(file_name,'r'):
        head,r,tail = [int(item) for item in line.split()]
        entity.add(head); entity.add(tail); rel.add(r+1)
        triples.append((head,r+1,tail))
    return entity,rel,triples

def load_alignment_pair(file_name):
    alignment_pair = []
    c = 0
    for line in open(file_name,'r'):
        e1,e2 = line.split()
        alignment_pair.append((int(e1),int(e2)))
    return alignment_pair

def get_matrix(triples,entity,rel):
        ent_size = max(entity)+1
        rel_size = (max(rel) + 1)
        print(ent_size,rel_size)
        adj_matrix = sp.lil_matrix((ent_size,ent_size))
        adj_features = sp.lil_matrix((ent_size,ent_size))
        radj = []
        rel_in = np.zeros((ent_size,rel_size))
        rel_out = np.zeros((ent_size,rel_size))
        
        for i in range(max(entity)+1):
            adj_features[i,i] = 1

        for h,r,t in triples:        
            adj_matrix[h,t] = 1; adj_matrix[t,h] = 1;
            adj_features[h,t] = 1; adj_features[t,h] = 1;
            radj.append([h,t,r]); radj.append([t,h,r+rel_size]); 
            rel_out[h][r] += 1; rel_in[t][r] += 1
            
        count = -1
        s = set()
        d = {}
        r_index,r_val = [],[]
        for h,t,r in sorted(radj,key=lambda x: x[0]*10e10+x[1]*10e5):
            if ' '.join([str(h),str(t)]) in s:
                r_index.append([count,r])
                r_val.append(1)
                d[count] += 1
            else:
                count += 1
                d[count] = 1
                s.add(' '.join([str(h),str(t)]))
                r_index.append([count,r])
                r_val.append(1)
        for i in range(len(r_index)):
            r_val[i] /= d[r_index[i][0]]
        
        rel_features = np.concatenate([rel_in,rel_out],axis=1)
        adj_features = normalize_adj(adj_features)
        rel_features = normalize_adj(sp.lil_matrix(rel_features))    
        return adj_matrix,r_index,r_val,adj_features,rel_features  

def load_data(lang, train_ratio = 0.3):
    entity1,rel1,triples1 = load_triples(lang + 'triples_1')
    entity2,rel2,triples2 = load_triples(lang + 'triples_2')
    val_rate=0.1
    if "_en" in lang:
        alignment_pair = load_alignment_pair(lang + 'ref_ent_ids')
        np.random.shuffle(alignment_pair)
        train_pair,valid_pair,dev_pair = alignment_pair[0:int(len(alignment_pair)*round(train_ratio-val_rate,1))],alignment_pair[int(len(alignment_pair)*round(train_ratio-val_rate,1)):int(len(alignment_pair)*train_ratio)],alignment_pair[int(len(alignment_pair)*train_ratio):]
    else:
        train_pair = load_alignment_pair(lang + 'sup_ent_ids')
        dev_pair = load_alignment_pair(lang + 'ref_ent_ids')
        ae_features = None
    
    adj_matrix,r_index,r_val,adj_features,rel_features = get_matrix(triples1+triples2,entity1.union(entity2),rel1.union(rel2))

    return np.array(train_pair),np.array(valid_pair),np.array(dev_pair),adj_matrix,np.array(r_index),np.array(r_val),adj_features, rel_features

In [16]:
data_ca="zh_en/"
data_dir = 'data/'+data_ca
train_pair,val_pair,dev_pair,adj_matrix,r_index,r_val,adj_features,rel_features = load_data(data_dir,train_ratio=0.30)
adj_matrix = np.stack(adj_matrix.nonzero(),axis = 1)
rel_matrix,rel_val = np.stack(rel_features.nonzero(),axis = 1),rel_features.data
ent_matrix,ent_val = np.stack(adj_features.nonzero(),axis = 1),adj_features.data

38960 3025


In [19]:
print(len(train_pair))

3000


In [20]:
print(len(val_pair))

1500


In [21]:
print(len(dev_pair))

10500


In [25]:
import torch

# 定义相似度矩阵 (3x3)
img_sim = torch.tensor([
    [0.32, 0.68, 1.04],
    [0.86, 1.82, 2.78],
    [1.40, 2.96, 4.52]
])

# 定义 get_topk_indices 函数
def get_topk_indices(M, K=1000):
    H, W = M.shape
    M_view = M.view(-1)
    vals, indices = M_view.topk(K)
    print("highest sim:", vals[0].item(), "lowest sim:", vals[-1].item())
    two_d_indices = torch.cat(((indices // W).unsqueeze(1), (indices % W).unsqueeze(1)), dim=1)
    return two_d_indices, vals

# 获取前 topk 元素及其索引
topk = 2
two_d_indices, vals = get_topk_indices(img_sim, topk * 100)

print("two_d_indices:")
print(two_d_indices)
print("vals:")
print(vals)


highest sim: 4.519999980926514 lowest sim: 0.3199999928474426
two_d_indices:
tensor([[2, 2],
        [2, 1],
        [1, 2],
        [1, 1],
        [2, 0],
        [0, 2],
        [1, 0],
        [0, 1],
        [0, 0]])
vals:
tensor([4.5200, 2.9600, 2.7800, 1.8200, 1.4000, 1.0400, 0.8600, 0.6800, 0.3200])


In [26]:
import torch

# 定义相似度矩阵 (3x3)
img_sim = torch.tensor([
    [0.32, 0.68, 1.04],
    [0.86, 1.82, 2.78],
    [1.40, 2.96, 4.52]
])

# 定义 get_topk_indices 函数
def get_topk_indices(M, K=1000):
    H, W = M.shape
    M_view = M.view(-1)
    if K > M_view.size(0):
        K = M_view.size(0)  # 确保 K 不超过矩阵中的元素数量
    vals, indices = M_view.topk(K)
    print("highest sim:", vals[0].item(), "lowest sim:", vals[-1].item())
    two_d_indices = torch.cat(((indices // W).unsqueeze(1), (indices % W).unsqueeze(1)), dim=1)
    return two_d_indices, vals

# 获取前 topk 元素及其索引
topk = 2
two_d_indices, vals = get_topk_indices(img_sim, topk * 100)

print("two_d_indices:")
print(two_d_indices)
print("vals:")
print(vals)


highest sim: 4.519999980926514 lowest sim: 0.3199999928474426
two_d_indices:
tensor([[2, 2],
        [2, 1],
        [1, 2],
        [1, 1],
        [2, 0],
        [0, 2],
        [1, 0],
        [0, 1],
        [0, 0]])
vals:
tensor([4.5200, 2.9600, 2.7800, 1.8200, 1.4000, 1.0400, 0.8600, 0.6800, 0.3200])


In [28]:
left_ents = [0, 2, 4]
right_ents = [1, 3, 5]
two_d_indices = torch.tensor([[2, 2], [1, 2], [2, 1], [1, 1], [2, 0], [1, 0], [0, 2], [0, 1], [0, 0]])
vals = torch.tensor([4.52, 2.78, 2.96, 1.82, 1.40, 0.86, 1.04, 0.68, 0.32])

used_inds = []
visual_links = []
count = 0
topk = 2

for index in range(len(two_d_indices)):
    ind = two_d_indices[index]
    if left_ents[ind[0]] in used_inds: continue
    if right_ents[ind[1]] in used_inds: continue
    used_inds.append(left_ents[ind[0]])
    used_inds.append(right_ents[ind[1]])
    visual_links.append((left_ents[ind[0]], right_ents[ind[1]], vals[index].cpu(), vals[index].cpu()))
    count += 1
    if count == topk:
        break


In [29]:
visual_links

[(4, 5, tensor(4.5200), tensor(4.5200)),
 (2, 3, tensor(1.8200), tensor(1.8200))]

In [30]:
class evaluateTorch:
    def __init__(self,  device, dev_pair,eval_batch_size=1024, k=10, batch=1024):
        self.device = device
        self.dev_pair = dev_pair
        self.eval_batch_size = eval_batch_size
        self.topk = k
        self.batch = batch

    def sim_results(self, Matrix_A, Matrix_B):
        # A x B.t
        A_sim = torch.mm(Matrix_A, Matrix_B.t())
        return A_sim



    def avg_results(self, Matrix_A):
        k = 10
        avg_results = torch.sum(torch.topk(Matrix_A, k=k)[0], dim=-1) / k
        return avg_results

    def CSLS_results(self, inputs):
        SxT_sim, TxS_avg, SxT_avg,ans_rank = inputs

        TxS_avg, SxT_avg = [torch.unsqueeze(m, dim=1) for m in [TxS_avg, SxT_avg]]
        sim = 2 * SxT_sim - SxT_avg - TxS_avg.t()
        # rank = torch.argsort(sim, dim=-1, descending=True)

        # targets = rank[:, :self.topk]
        # values = torch.gather(sim, 1, targets)

        #     rank = tf.argsort(-sim,axis=-1)
        #     results = tf.where(tf.equal(rank,K.cast(K.tile(ans_rank,[1,len(self.dev_pair)]),dtype="int32")))
        #     return K.expand_dims(results,axis=0)
        # results = torch.where(torch.equal(rank, torch.tensor(ans_rank).unsqueeze(0).repeat(1, len(self.dev_pair)).int()))
        rank = torch.argsort(-sim, dim=-1)
        results = torch.where(torch.eq(rank, torch.tensor(ans_rank).unsqueeze(1).repeat(1, len(self.dev_pair)).to(self.device)))
        return results[1]
        # return torch.unsqueeze(results[1], dim=0)

        # return targets.cpu().numpy(), values.cpu().numpy()

    def CSLS_rank(self, inputs):
        SxT_sim, TxS_avg, SxT_avg,ans_rank = inputs

        TxS_avg, SxT_avg = [torch.unsqueeze(m, dim=1) for m in [TxS_avg, SxT_avg]]
        sim = 2 * SxT_sim - SxT_avg - TxS_avg.t()
        # rank = torch.argsort(sim, dim=-1, descending=True)

        # targets = rank[:, :self.topk]
        # values = torch.gather(sim, 1, targets)

        #     rank = tf.argsort(-sim,axis=-1)
        #     results = tf.where(tf.equal(rank,K.cast(K.tile(ans_rank,[1,len(self.dev_pair)]),dtype="int32")))
        #     return K.expand_dims(results,axis=0)
        # results = torch.where(torch.equal(rank, torch.tensor(ans_rank).unsqueeze(0).repeat(1, len(self.dev_pair)).int()))
        rank = torch.argsort(-sim, dim=-1)
     
        return rank[:,0]

    def CSLS_cal(self, sourceVec, targetVec,m1,m2,evaluate=True):
        batch_size = self.eval_batch_size
        SxT_sim = []
        TxS_sim=[]
        for epoch in range(len(sourceVec) // batch_size + 1):
            SxT_sim.append(self.sim_results(sourceVec[epoch * batch_size:(epoch + 1) * batch_size], targetVec))
            TxS_sim.append(self.sim_results(targetVec[epoch * batch_size:(epoch + 1) * batch_size], sourceVec))

        alpha=0.2
        # if evaluate:

        for epoch in range(len(sourceVec) // batch_size + 1):
            SxT_sim[epoch]=SxT_sim[epoch]*(1-alpha)+m1[epoch * batch_size:(epoch + 1) * batch_size,:]*alpha
            TxS_sim[epoch]=TxS_sim[epoch]*(1-alpha)+m2[epoch * batch_size:(epoch + 1) * batch_size,:]*alpha


        SxT_avg = []
        TxS_avg = []
        for epoch in range(len(sourceVec) // batch_size + 1):
            SxT_avg.append(self.avg_results(SxT_sim[epoch]))
            TxS_avg.append(self.avg_results(TxS_sim[epoch]))

        # SxT_sim = self.sim_results(sourceVec, targetVec)
        # SxT_sim = SxT_sim*(1-alpha)+m*alpha

        # SxT_avg = self.avg_results(SxT_sim)

        # TxS_sim = self.sim_results(targetVec, sourceVec)
        # TxS_sim = TxS_sim*(1-alpha)+m*alpha

        # TxS_avg = self.avg_results(TxS_sim)


        if evaluate:
            targets = np.empty((0, self.topk), int)
            targets=[]
            for epoch in range(len(sourceVec) // batch_size + 1):
                ans_rank = np.array([i for i in range(epoch * batch_size,min((epoch+1) * batch_size,len(sourceVec)))])
                temp_targets = self.CSLS_results([SxT_sim[epoch].to(device=self.device),torch.cat((TxS_avg),dim=0).to(device=self.device), SxT_avg[epoch].to(device=self.device),ans_rank])
                # targets = np.concatenate((targets, temp_targets), axis=0)
                targets.append(temp_targets)
                # values = np.concatenate((values, temp_values), axis=0)
            return torch.cat((targets),0)
        else :
            l_rank,r_rank = [],[]
            for epoch in range(len(sourceVec) // batch_size + 1):
                ans_rank = np.array([i for i in range(epoch * batch_size,min((epoch+1) * batch_size,len(sourceVec)))])
                l_rank.append(self.CSLS_rank([SxT_sim[epoch].to(device=self.device), torch.cat((TxS_avg),dim=0).to(device=self.device), SxT_avg[epoch].to(device=self.device),ans_rank]))
                r_rank.append(self.CSLS_rank([TxS_sim[epoch].to(device=self.device), torch.cat((SxT_avg),dim=0).to(device=self.device), TxS_avg[epoch].to(device=self.device),ans_rank]))

            # return np.concatenate(r_rank,axis=1)[0],np.concatenate(l_rank,axis=1)[0] 
            return  torch.cat((r_rank),dim=0).cpu().numpy(),torch.cat((l_rank),dim=0).cpu().numpy()

In [31]:
Lvec = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
Rvec = torch.tensor([[2.0, 1.0], [4.0, 3.0], [6.0, 5.0]])
img_sim_dev_boot = torch.tensor([[0.9, 0.8, 0.7], [0.6, 0.5, 0.4], [0.3, 0.2, 0.1]])
dev_pair = [(0, 0), (1, 1), (2, 2)]


In [38]:
import torch
import numpy as np

class evaluateTorch:
    def __init__(self, device, dev_pair, eval_batch_size=1024, k=10, batch=1024):
        self.device = device
        self.dev_pair = dev_pair
        self.eval_batch_size = eval_batch_size
        self.topk = k
        self.batch = batch

    def sim_results(self, Matrix_A, Matrix_B):
        # A x B.t
        A_sim = torch.mm(Matrix_A, Matrix_B.t())
        return A_sim

    def avg_results(self, Matrix_A):
        k = min(self.topk, Matrix_A.size(1))  # 确保 k 不超过 Matrix_A 的大小
        avg_results = torch.sum(torch.topk(Matrix_A, k=k)[0], dim=-1) / k
        return avg_results

    def CSLS_results(self, inputs):
        SxT_sim, TxS_avg, SxT_avg, ans_rank = inputs

        TxS_avg, SxT_avg = [torch.unsqueeze(m, dim=1) for m in [TxS_avg, SxT_avg]]
        sim = 2 * SxT_sim - SxT_avg - TxS_avg.t()

        rank = torch.argsort(-sim, dim=-1)
        results = torch.where(torch.eq(rank, torch.tensor(ans_rank).unsqueeze(1).repeat(1, len(self.dev_pair)).to(self.device)))
        return results[1]

    def CSLS_rank(self, inputs):
        SxT_sim, TxS_avg, SxT_avg, ans_rank = inputs

        TxS_avg, SxT_avg = [torch.unsqueeze(m, dim=1) for m in [TxS_avg, SxT_avg]]
        sim = 2 * SxT_sim - SxT_avg - TxS_avg.t()

        rank = torch.argsort(-sim, dim=-1)
        return rank[:, 0]

    def CSLS_cal(self, sourceVec, targetVec, m1, m2, evaluate=True):
        batch_size = self.eval_batch_size
        SxT_sim = []
        TxS_sim = []
        for epoch in range(len(sourceVec) // batch_size + 1):
            SxT_sim.append(self.sim_results(sourceVec[epoch * batch_size:(epoch + 1) * batch_size], targetVec))
            TxS_sim.append(self.sim_results(targetVec[epoch * batch_size:(epoch + 1) * batch_size], sourceVec))

        alpha = 0.2
        for epoch in range(len(sourceVec) // batch_size + 1):
            SxT_sim[epoch] = SxT_sim[epoch] * (1 - alpha) + m1[epoch * batch_size:(epoch + 1) * batch_size, :] * alpha
            TxS_sim[epoch] = TxS_sim[epoch] * (1 - alpha) + m2[epoch * batch_size:(epoch + 1) * batch_size, :] * alpha

        SxT_avg = []
        TxS_avg = []
        for epoch in range(len(sourceVec) // batch_size + 1):
            SxT_avg.append(self.avg_results(SxT_sim[epoch]))
            TxS_avg.append(self.avg_results(TxS_sim[epoch]))

        if evaluate:
            targets = []
            for epoch in range(len(sourceVec) // batch_size + 1):
                ans_rank = np.array([i for i in range(epoch * batch_size, min((epoch + 1) * batch_size, len(sourceVec)))])
                temp_targets = self.CSLS_results([SxT_sim[epoch].to(device=self.device), torch.cat((TxS_avg), dim=0).to(device=self.device), SxT_avg[epoch].to(device=self.device), ans_rank])
                targets.append(temp_targets)
            return torch.cat((targets), 0)
        else:
            l_rank, r_rank = [], []
            for epoch in range(len(sourceVec) // batch_size + 1):
                ans_rank = np.array([i for i in range(epoch * batch_size, min((epoch + 1) * batch_size, len(sourceVec)))])
                l_rank.append(self.CSLS_rank([SxT_sim[epoch].to(device=self.device), torch.cat((TxS_avg), dim=0).to(device=self.device), SxT_avg[epoch].to(device=self.device), ans_rank]))
                r_rank.append(self.CSLS_rank([TxS_sim[epoch].to(device=self.device), torch.cat((SxT_avg), dim=0).to(device=self.device), TxS_avg[epoch].to(device=self.device), ans_rank]))

            return  torch.cat((r_rank),dim=0).cpu().numpy(),torch.cat((l_rank),dim=0).cpu().numpy()

# 假设的数据
Lvec = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
Rvec = torch.tensor([[2.0, 1.0], [4.0, 3.0], [6.0, 5.0]])
img_sim_dev_boot = torch.tensor([[0.9, 0.8, 0.7], [0.6, 0.5, 0.4], [0.3, 0.2, 0.1]])
dev_pair = [(0, 0), (1, 1), (2, 2)]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 创建 evaluateTorch 实例
evaluater_torch = evaluateTorch(device, dev_pair)

# 执行 CSLS 计算
A,B = evaluater_torch.CSLS_cal(Lvec, Rvec, img_sim_dev_boot, img_sim_dev_boot.t(), evaluate=False)

print("CSLS 结果:")
print(A)
print(B)


CSLS 结果:
[0 2 2]
[0 2 2]


In [42]:
import torch

# 创建一个 300 维的向量
vector = torch.randn(100,300)

# 分离出一个新的张量，不会记录梯度
detached_vector = vector.detach()

# 打印张量以验证
print("原始向量:", vector)
print("分离后的向量:", detached_vector)


原始向量: tensor([[ 1.6935,  1.2172, -0.0474,  ..., -0.8397,  0.7084, -0.1599],
        [ 0.8791, -1.3297,  0.8217,  ...,  0.4250, -0.0378, -1.2609],
        [ 0.5091, -1.7833, -0.8747,  ..., -1.0997,  0.6612,  0.4733],
        ...,
        [-1.2154,  0.1178,  2.8064,  ..., -0.2860,  1.5074, -1.5766],
        [-0.9026,  0.2341, -2.3404,  ...,  0.1798,  2.3059, -0.9629],
        [ 1.0292, -0.4149,  0.9449,  ..., -0.6257,  1.2328,  0.6047]])
分离后的向量: tensor([[ 1.6935,  1.2172, -0.0474,  ..., -0.8397,  0.7084, -0.1599],
        [ 0.8791, -1.3297,  0.8217,  ...,  0.4250, -0.0378, -1.2609],
        [ 0.5091, -1.7833, -0.8747,  ..., -1.0997,  0.6612,  0.4733],
        ...,
        [-1.2154,  0.1178,  2.8064,  ..., -0.2860,  1.5074, -1.5766],
        [-0.9026,  0.2341, -2.3404,  ...,  0.1798,  2.3059, -0.9629],
        [ 1.0292, -0.4149,  0.9449,  ..., -0.6257,  1.2328,  0.6047]])


In [44]:
detached_vector.shape

torch.Size([100, 300])

In [45]:
import torch
import numpy as np

def CSLS_rank(inputs):
    SxT_sim, TxS_avg, SxT_avg, ans_rank = inputs

    # 将 TxS_avg 和 SxT_avg 变为列向量
    TxS_avg, SxT_avg = [torch.unsqueeze(m, dim=1) for m in [TxS_avg, SxT_avg]]

    # 计算 CSLS 相似度矩阵
    sim = 2 * SxT_sim - SxT_avg - TxS_avg.t()

    # 对每行进行降序排序，返回排序后的索引
    rank = torch.argsort(-sim, dim=-1)

    # 返回每行中相似度最高的目标索引
    return rank[:, 0]

# 示例数据
SxT_sim = torch.tensor([[0.9, 0.8, 0.7],
                        [0.6, 0.5, 0.4],
                        [0.3, 0.2, 0.1]])

TxS_avg = torch.tensor([0.6, 0.5, 0.4])
SxT_avg = torch.tensor([0.8, 0.5, 0.2])
ans_rank = torch.tensor([0, 1, 2])  # 示例答案排名

# 输入数据
inputs = (SxT_sim, TxS_avg, SxT_avg, ans_rank)

# 计算排名
top_rank = CSLS_rank(inputs)

print("Top rank indices for each source:")
print(top_rank)


Top rank indices for each source:
tensor([0, 0, 0])
